## Load in datasets

This will take a minute or two.

In [1]:
from cornell import Cornell

dataset = Cornell()
(train_data, train_labels), (test_data, test_labels) = dataset.load_data()

Using TensorFlow backend.


Also grab the index of encodings to genres.

In [2]:
idx_to_genre = dataset.get_idx_to_genre()

In [3]:
import numpy as np

In [5]:
num_words = 10000
num_genres = len(idx_to_genre)

# From https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/3.5-classifying-movie-reviews.ipynb
def vectorize_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data, num_words)
x_test = vectorize_sequences(test_data, num_words)

y_train = vectorize_sequences(train_labels, num_genres)
y_test = vectorize_sequences(test_labels, num_genres)

In [ ]:
from keras import models, layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(num_words,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(y_train.shape[1], activation='sigmoid'))

In [224]:
from keras import models, layers, backend
from keras.constraints import maxnorm

In [292]:
model = models.Sequential()
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_genres, activation='sigmoid'))

In [246]:
backend.clear_session()

In [296]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [297]:
model.fit(x_train, y_train, epochs=20, batch_size=16, validation_split=0.1)

Train on 415 samples, validate on 47 samples
Epoch 1/10
415/415 [==============================] - 2s 4ms/step - loss: 0.1933 - acc: 0.9214 - val_loss: 0.2499 - val_acc: 0.8980
Epoch 2/10
415/415 [==============================] - 0s 961us/step - loss: 0.1785 - acc: 0.9274 - val_loss: 0.2656 - val_acc: 0.9069
Epoch 3/10
415/415 [==============================] - 0s 959us/step - loss: 0.1735 - acc: 0.9282 - val_loss: 0.2512 - val_acc: 0.9034
Epoch 4/10
415/415 [==============================] - 0s 950us/step - loss: 0.1618 - acc: 0.9336 - val_loss: 0.2556 - val_acc: 0.9043
Epoch 5/10
415/415 [==============================] - 0s 956us/step - loss: 0.1609 - acc: 0.9343 - val_loss: 0.2443 - val_acc: 0.9043
Epoch 6/10
415/415 [==============================] - 0s 888us/step - loss: 0.1498 - acc: 0.9391 - val_loss: 0.2564 - val_acc: 0.9060
Epoch 7/10
415/415 [==============================] - 0s 952us/step - loss: 0.1438 - acc: 0.9414 - val_loss: 0.2496 - val_acc: 0.8998
Epoch 8/10
415/415 

In [298]:
results = model.evaluate(x_test, y_test)
results

155/155 [==============================] - 0s 167us/step


[0.24686938437723344, 0.9069892425690927]

In [153]:
predictions = model.predict(x_test)
predictions[predictions>=0.5] = 1
predictions[predictions<0.5] = 0

array([0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [154]:
entries = len(y_test)
total = num_genres * entries

correct = 0
for i in range(entries):
    for j in range(num_genres):
        correct += int(predictions[i][j] == y_test[i][j])

print(str(correct / total))

0.9048387096774193


In [44]:
predictions = model.predict(y_test)

ValueError: Error when checking input: expected dense_1_input to have shape (10000,) but got array with shape (24,)